In [14]:
import os
import json
import pandas as pd
import traceback
import openai
import langchain 



In [15]:
from langchain.chat_models import ChatOpenAI

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [21]:
from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv('OPENAI_API_KEY ')
llm = ChatOpenAI(model_name = "gpt-3.5-turbo",api_key=KEY)

In [53]:
TEMPLATE = """
Text: {text}

As an expert in MCQ generation, your task is to create a quiz consisting of {number} multiple-choice questions for students studying {subject}. The questions should be crafted in a {tone} manner and reflect the context provided. Each question must have one correct answer and three incorrect options.

Ensure your responses are structured as follows, using JSON format for clarity:
{response_json}
"""

response_json_template = {
    "questions": [
        {
            "question": "What is the capital of France?",
            "options": ["a. Paris", "b. London", "c. Rome", "d. Madrid"],
            "correct_option": "a"
        }
    ]
}

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone",'response_json'],
    template=TEMPLATE
)


In [54]:
json.dumps(response_json_template)

'{"questions": [{"question": "What is the capital of France?", "options": ["a. Paris", "b. London", "c. Rome", "d. Madrid"], "correct_option": "a"}]}'

In [55]:
quiz_generator_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key='quiz_questions', verbose=True)


In [56]:
EVALUATION_TEMPLATE = """
As an expert in English grammar and writing, you have been presented with a Multiple Choice Quiz intended for {subject} students. Your task is to evaluate the complexity of the questions and provide a comprehensive analysis of the quiz. Offer concise feedback, limited to 50 words per question, addressing the cognitive and analytical demands posed by the quiz. If necessary, suggest modifications to the questions that will better align with the students' abilities.

Quiz MCQs:
{quiz_questions}

Based on your expertise as an English Writer, provide your critique and suggested enhancements for the quiz above:
"""

quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz_questions"],
    template=EVALUATION_TEMPLATE
)


In [57]:
quiz_review_chain = LLMChain(llm= llm, prompt=quiz_evaluation_prompt, verbose=True, output_key='quiz_review')


In [58]:
final_chain = SequentialChain(chains=[quiz_generator_chain,quiz_review_chain], input_variables=["text", "number", "subject", "tone",'response_json'],output_variables=['quiz_questions','quiz_review'],verbose=True)

In [59]:
file_path = r'/Users/loke/Library/CloudStorage/Dropbox/Project/LLM Projects/LLM_MCQ_Generator/data.txt'
with open(file_path,'r') as file:
    TEXT = file.read()



In [60]:
NUMBER = 3
SUBJECT = 'Finanace'
TONE = 'simple'

#This helps to track the token usage in openai
with get_openai_callback() as cb:
    response = final_chain({'text':TEXT,
                            'number':NUMBER,
                            'subject':SUBJECT,
                            'tone':TONE,
                            'response_json':json.dumps(response_json_template)
                            })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Introduction to Bank Accounts
A bank account is typically the first financial account that you’ll open. Bank accounts can hold and build the money you'll need for major purchases and life events. Here’s some background on bank accounts and why they are step one in creating a stable financial future.

Why Do I Need a Bank Account?
Though the majority of Americans do have bank accounts, 6% of households in the United States still don’t have one.
2
 Why is it so important to open a bank account? Because it’s safer than holding cash. Assets held in a bank are harder to steal, and in the U.S., they’re generally insured by the Federal Deposit Insurance Corporation (FDIC).
3
 That means you should always have access to your cash, even if every customer decided to withdraw their money at the same time.

Many financial transactions require you to have a bank account to:

Use a debit or c

In [61]:
response.get('quiz_questions')

'\n{\n  "questions": [\n    {\n      "question": "What is the purpose of opening a bank account?",\n      "options": [\n        "a. To hold and build money for major purchases and life events",\n        "b. To store cash at home",\n        "c. To invest in stocks and bonds",\n        "d. To hide money from the government"\n      ],\n      "correct_option": "a"\n    },\n    {\n      "question": "What is the main difference between a debit card and a credit card?",\n      "options": [\n        "a. Debit cards allow you to borrow money, while credit cards do not",\n        "b. Credit cards take money directly from your bank account, while debit cards do not",\n        "c. Credit cards help you build a credit history, while debit cards do not",\n        "d. Debit cards charge interest on unpaid balances, while credit cards do not"\n      ],\n      "correct_option": "c"\n    },\n    {\n      "question": "What is APR in relation to credit cards?",\n      "options": [\n        "a. Annual Paym

In [62]:
quiz_questions_final = response.get('quiz_questions')
quiz_questions_final = json.loads(quiz_questions_final)
quiz_questions_final

{'questions': [{'question': 'What is the purpose of opening a bank account?',
   'options': ['a. To hold and build money for major purchases and life events',
    'b. To store cash at home',
    'c. To invest in stocks and bonds',
    'd. To hide money from the government'],
   'correct_option': 'a'},
  {'question': 'What is the main difference between a debit card and a credit card?',
   'options': ['a. Debit cards allow you to borrow money, while credit cards do not',
    'b. Credit cards take money directly from your bank account, while debit cards do not',
    'c. Credit cards help you build a credit history, while debit cards do not',
    'd. Debit cards charge interest on unpaid balances, while credit cards do not'],
   'correct_option': 'c'},
  {'question': 'What is APR in relation to credit cards?',
   'options': ['a. Annual Payment Rate',
    'b. Additional Purchase Requirement',
    'c. Average Percentage Return',
    'd. Annual Percentage Rate'],
   'correct_option': 'd'}]}

In [65]:
# Normalize the JSON data into a flat table
quiz_questions_final = pd.json_normalize(quiz_questions_final['questions'])

quiz_questions_final['options'] = quiz_questions_final['options'].apply(lambda opts: '\n'.join(opts))

quiz_questions_final

,question,options,correct_option
0,What is the purpose of opening a bank account?,a. To hold and build money for major purchases...,a
1,What is the main difference between a debit ca...,"a. Debit cards allow you to borrow money, whil...",c
2,What is APR in relation to credit cards?,a. Annual Payment Rate\nb. Additional Purchase...,d
